In [1]:
pip install nltk

In [2]:
pip install numpy

In [3]:
pip install tflearn

     |████████████████████████████████| 112kB 7.6MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp37-none-any.whl size=127300 sha256=1317e46bfdfe1fb0d248e41eb9f706eca4334665d7608b941a377f4277dc43d6
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [4]:
pip install tensorflow

In [5]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [6]:
import json
import random
from typing import List
from numpy.core.defchararray import mod
import tensorflow as tf
from tensorflow.python.ops.variables import model_variables
import tflearn
import numpy
import nltk
import pickle
from nltk.stem.api import StemmerI
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

Instructions for updating:
non-resource variables are not supported in the long term


In [8]:
with open("chatsss.json") as file:
    data = json.load(file)

In [9]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
try:
    with open("data.oke", "rb") as f:
        words, labels, training, output = oke.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.oke", "wb") as f:
        pickle.dump((words, labels, training, output), f)


In [11]:
tf.compat.v1.reset_default_graph()

In [12]:
try:
    model.load('model.tflearn')
except:
    tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
# input data in full neural layers and connect to neural network
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [13]:
model = tflearn.DNN(net)

In [14]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

Training Step: 4999  | total loss: 0.00668 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.00668 - acc: 1.0000 -- iter: 32/33
Training Step: 5000  | total loss: 0.00752 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.00752 - acc: 1.0000 -- iter: 33/33
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [15]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)

In [16]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)

In [17]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
        print(random.choice(responses))

In [19]:
chat()

Start talking with the bot (type quit to stop)!
You: quit
